In [1]:
%matplotlib inline

# Force GeoPandas to use Shapely instead of PyGEOS
# In a future release, GeoPandas will switch to using Shapely by default.
import os
os.environ['USE_PYGEOS'] = '0'

import datacube
import numpy as np
import xarray as xr
import subprocess as sp
import geopandas as gpd
from odc.io.cgroups import get_cpu_quota
from datacube.utils.geometry import assign_crs

from deafrica_tools.plotting import map_shapefile
from deafrica_tools.bandindices import calculate_indices
from deafrica_tools.classification import collect_training_data

In [2]:
path = 'Western_Cape.geojson' 
field = 'Class'

In [3]:
ncpus=round(get_cpu_quota())
print('ncpus = '+str(ncpus))

ncpus = 4


In [4]:
#open shapefile and ensure its in WGS84 coordinates
input_data = gpd.read_file(path)
#.to_crs('epsg:6933')

# Plot first five rows
input_data.head()

,Class,name,geometry
0,0,Water,POINT (1826749.345 -4109627.495)
1,0,Water,POINT (1828513.056 -4112615.738)
2,0,Water,POINT (1820874.448 -4112834.019)
3,0,Water,POINT (1823271.605 -4111155.852)
4,0,Water,POINT (1870872.515 -4098289.672)


In [5]:
print(input_data.crs)

EPSG:6933


In [6]:
#CHANGE GEOMETRY TYPE TO POLYGON

# Set a flag to convert to polygons:
use_polygons = True

if use_polygons:
   # Buffer geometry to get a square - only if trying to sample multiple pixels
    buffer_radius_m = 15
    input_data.geometry = input_data.geometry.buffer(buffer_radius_m, cap_style=3)

# Plot first five rows
input_data.head()

,Class,name,geometry
0,0,Water,"POLYGON ((1826764.345 -4109612.495, 1826764.34..."
1,0,Water,"POLYGON ((1828528.056 -4112600.738, 1828528.05..."
2,0,Water,"POLYGON ((1820889.448 -4112819.019, 1820889.44..."
3,0,Water,"POLYGON ((1823286.605 -4111140.852, 1823286.60..."
4,0,Water,"POLYGON ((1870887.515 -4098274.672, 1870887.51..."


In [7]:
# Plot training data in an interactive map
map_shapefile(input_data, attribute=field)

Label(value='')

Map(center=[-33.923116662035305, 19.976780329869264], controls=(ZoomControl(options=['position', 'zoom_in_text…

In [8]:
#set up our inputs to collect_training_data
zonal_stats = 'mean'

# Set up the inputs for the ODC query
time = ('2019')

S_measurements =  ['blue','green','red','nir','swir_1','swir_2']

resolution = (-10,10)

output_crs='epsg:6933'

In [9]:
query = {
    'time': time,
    'measurements': S_measurements,
    'resolution': resolution,
    'output_crs': output_crs
}

In [10]:
from datacube.testutils.io import rio_slurp_xarray

def feature_layers(query):
    #connect to the datacube
    dc = datacube.Datacube(app='feature_layers')
    
    #load s2 annual geomedian
    ds = dc.load(product='gm_s2_annual',
                 **query)
    
    #calculate some band indices
    da = calculate_indices(ds,
                           index=['NDVI', 'LAI', 'MNDWI'],
                           drop=False,
                           satellite_mission='s2')
    
    #add slope dataset
    url_slope = "https://deafrica-input-datasets.s3.af-south-1.amazonaws.com/srtm_dem/srtm_africa_slope.tif"
    slope = rio_slurp_xarray(url_slope, gbox=ds.geobox)
    slope = slope.to_dataset(name='slope')
    
    #merge results into single dataset 
    result = xr.merge([da, slope],compat='override')

    return result.squeeze()

In [11]:
column_names, model_input = collect_training_data(
                                    gdf=input_data,
                                    dc_query=query,
                                    ncpus=ncpus,
                                    field=field,
                                    zonal_stats=zonal_stats,
                                    feature_func=feature_layers
                                    )

Taking zonal statistic: mean


  0%|          | 0/2700 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
print(column_names)

In [ ]:
print(np.array_str(model_input, precision=2, suppress_small=True))

In [ ]:
#set the name and location of the output file
output_file = "S_training_data(1).txt"

In [ ]:
#grab all columns
model_col_indices = [column_names.index(var_name) for var_name in column_names]
#Export files to disk
np.savetxt(output_file, model_input[:, model_col_indices], header=" ".join(column_names), fmt="%4f")